In [1]:
!pip install torch torchvision ninja > /dev/null

In [ ]:
!pip install ninja

In [ ]:
from pathlib import Path
import torch
from torchvision.io import read_image, write_png
from torch.utils.cpp_extension import load_inline

### mean_filter_kernel

In [96]:
cuda_source = """
#include <c10/cuda/CUDAException.h>
#include <c10/cuda/CUDAStream.h>


__global__
void mean_filter_kernel(unsigned char* output, unsigned char* input, int width, int height, int radius) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int channel = threadIdx.z;

    int baseOffset = channel * height * width;
    if (col < width && row < height) {

        int pixVal = 0;
        int pixels = 0;

        for (int blurRow=-radius; blurRow <= radius; blurRow += 1) {
            for (int blurCol=-radius; blurCol <= radius; blurCol += 1) {
                int curRow = row + blurRow;
                int curCol = col + blurCol;
                if (curRow >= 0 && curRow < height && curCol >=0 && curCol < width) {
                    pixVal += input[baseOffset + curRow * width + curCol];
                    pixels += 1;
                }
            }
        }

        output[baseOffset + row * width + col] = (unsigned char)(pixVal / pixels);
    }
}


// helper function for ceiling unsigned integer division
inline unsigned int cdiv(unsigned int a, unsigned int b) {
  return (a + b - 1) / b;
}

// this is a cpp function, which is called as cpp_source
torch::Tensor mean_filter(torch::Tensor image, int radius) {
    assert(image.device().type() == torch::kCUDA);
    assert(image.dtype() == torch::kByte);
    assert(radius > 0);

    const auto channels = image.size(0);
    const auto height = image.size(1);
    const auto width = image.size(2);

    auto result = torch::empty_like(image);

    dim3 threads_per_block(16, 16, channels);
    dim3 number_of_blocks(
        cdiv(width, threads_per_block.x),
        cdiv(height, threads_per_block.y)
    );

    mean_filter_kernel<<<number_of_blocks, threads_per_block, 0, torch::cuda::getCurrentCUDAStream()>>>(
        result.data_ptr<unsigned char>(),
        image.data_ptr<unsigned char>(),
        width,
        height,
        radius
    );

    // check CUDA error status (calls cudaGetLastError())
    C10_CUDA_KERNEL_LAUNCH_CHECK();

    return result;
}"""

In [9]:
# deleting files with python in current path.
import os

def clear_cudafiles(path_str):
  """Clears the files alone in pwd"""
  objs = os.listdir(path_str)
  for obj in objs:
    if os.path.isfile(os.path.join(path_str, obj)):
      os.remove(os.path.join(path_str, obj))
  print("Files deleted. Refresh...")

In [10]:
def compile_extension(cuda_source: str, cpp_source: str,
                      ext_name: str, call_funcs: list,
                      build_dir='.'):
  clear_cudafiles(path_str=build_dir)
  your_extension = load_inline(
      name=ext_name,
      cpp_sources=cpp_source,
      cuda_sources=cuda_source,
      functions=call_funcs,
      with_cuda=True,
      extra_cuda_cflags=['-O2 -arch=sm_75 -std=c++20'],  # sm_75 is chosen for turning arch
      build_directory='.',
      )
  return your_extension
# https://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html?highlight=virtual%20arch#virtual-architectures
# Note the cuda C++ guide is different to NVCC guide provided above

In [97]:
cpp_source = "torch::Tensor mean_filter(torch::Tensor image, int radius);"

rgb_extension = compile_extension(cuda_source=cuda_source, cpp_source=cpp_source, ext_name="rgb_blur", call_funcs=['mean_filter'])

Files deleted. Refresh...


In [19]:
x = read_image("Grace_Hopper.jpg").contiguous().cuda()
assert x.dtype == torch.uint8
print("Input image:", x.shape, x.dtype)

Input image: torch.Size([3, 606, 517]) torch.uint8


In [20]:
y = rgb_extension.mean_filter(x, 8)

print("Output image:", y.shape, y.dtype)

write_png(y.cpu(), "output.png")

Output image: torch.Size([3, 606, 517]) torch.uint8


### vector add kernel

In [74]:
cuda_source = """
#include <cuda.h>
#include <stdio.h>

// compute vector sum C = A + B
// each thread peforms one pair-wise addition

__global__ void vecAddKernel(float *A, float *B, float *C, int n) {
  printf("threadIdx: %d ", threadIdx.x);
  printf("blockIdx: %d", blockIdx.x);
  printf("blockDim: %d", blockDim.x);
  int i = threadIdx.x + blockDim.x * blockIdx.x;
  if (i < n) {
    C[i] = A[i] + B[i];
  }
}

// https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api

/*
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort = true) {
  if (code != cudaSuccess) {
    fprintf(stderr, "GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
    if (abort) {
      exit(code);
    }
  }
}
*/

inline unsigned int cdiv(unsigned int a, unsigned int b) {
  return (a + b - 1) / b;
}

// following is the cpp function
void vecAdd(float *A, float *B, float *C, int n) {
  float *A_d, *B_d, *C_d;
  size_t size = n * sizeof(float);

  cudaMalloc((void **)&A_d, size);
  cudaMalloc((void **)&B_d, size);
  cudaMalloc((void **)&C_d, size);

  cudaMemcpy(A_d, A, size, cudaMemcpyHostToDevice);
  cudaMemcpy(B_d, B, size, cudaMemcpyHostToDevice);

  const unsigned int numThreads = 256;
  unsigned int numBlocks = cdiv(n, numThreads);

  vecAddKernel<<<numBlocks, numThreads>>>(A_d, B_d, C_d, n);
  // gpuErrchk(cudaPeekAtLastError());
  // gpuErrchk(cudaDeviceSynchronize());

  cudaMemcpy(C, C_d, size, cudaMemcpyDeviceToHost);

  cudaFree(A_d);
  cudaFree(B_d);
  cudaFree(C_d);
}

int main() {
  const int n = 1000;
  float A[n];
  float B[n];
  float C[n];

  // generate some dummy vectors to add
  for (int i = 0; i < n; i += 1) {
    A[i] = float(i);
    B[i] = A[i] / 1000.0f;
  }
  std::cout << "created input vectors";
  vecAdd(A, B, C, n);
  std::cout << "Call to vecAdd done. Printing results";

  // print result
  for (int i = 0; i < n; i += 1) {
    if (i > 0) {
      std::cout << ", ";
      if (i % 10 == 0) {
        std::cout << std::endl;
      }
    }
    std::cout << C[i];
  }
  return 100;
}"""

SOME Observation when compiling through the load_inline function

- printf() and "\n" needs to be avoided, because they get parsed differently when cuda.cu file is created below. Usage of std::out / std::endl will be better option.

- The function that calls CUDA Kernel is embedded inside a C++ function, which is to be called with correct inputs from python environment.

- function writes out the cuda.cu file, along with seperate main.cpp file which contains the below code.

  ```
  #include <torch/extension.h>

  torch::Tensor mean_filter(torch::Tensor image, int radius);

  PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
    m.def("mean_filter", torch::wrap_pybind_function(mean_filter), "mean_filter");
  }
  ```
- When the extension needs to be used, the name of the variable containing the extension object, followed by the .func_name in the above code snippet needs to be used.

In [75]:
cpp_source = "int main();"

In [76]:
vector_add_ext = compile_extension(cuda_source=cuda_source,
                                   cpp_source=cpp_source,
                                   ext_name='vector_add',
                                   call_funcs=['main'])
# the compiling extension failed, however the cuda code itself ran successfully.

Files deleted. Refresh...


In [78]:
!ls

build.ninja  cuda.cu  cuda.cuda.o  main.cpp  main.o  sample_data  vector_add_v7.so


In [90]:
!nvcc -o vector_addition -arch=sm_75 -O2 vector_addition.cu

In [ ]:
!nvprof ./vector_addition

In [ ]:
!ncu -o vecadd_profile vector_addition

In [ ]:
!./vector_addition

### rgb_to_grayscale kernel

In [6]:
cuda_source = """
#include <c10/cuda/CUDAException.h>
#include <c10/cuda/CUDAStream.h>


__global__
void rgb_to_grayscale_kernel(unsigned char* output, unsigned char* input, int width, int height) {
    const int channels = 3;

    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (col < width && row < height) {
        int outputOffset = row * width + col;
        int inputOffset = (row * width + col) * channels;

        unsigned char r = input[inputOffset + 0];   // red
        unsigned char g = input[inputOffset + 1];   // green
        unsigned char b = input[inputOffset + 2];   // blue

        output[outputOffset] = (unsigned char)(0.21f * r + 0.71f * g + 0.07f * b);
    }
}


// helper function for ceiling unsigned integer division
inline unsigned int cdiv(unsigned int a, unsigned int b) {
  return (a + b - 1) / b;
}


torch::Tensor rgb_to_grayscale(torch::Tensor image) {
    assert(image.device().type() == torch::kCUDA);
    assert(image.dtype() == torch::kByte);

    const auto height = image.size(0);
    const auto width = image.size(1);

    auto result = torch::empty({height, width, 1}, torch::TensorOptions().dtype(torch::kByte).device(image.device()));

    dim3 threads_per_block(16, 16);     // using 256 threads per block
    dim3 number_of_blocks(cdiv(width, threads_per_block.x),
                          cdiv(height, threads_per_block.y));

    rgb_to_grayscale_kernel<<<number_of_blocks, threads_per_block, 0, torch::cuda::getCurrentCUDAStream()>>>(
        result.data_ptr<unsigned char>(),
        image.data_ptr<unsigned char>(),
        width,
        height
    );

    // check CUDA error status (calls cudaGetLastError())
    C10_CUDA_KERNEL_LAUNCH_CHECK();

    return result;
}
"""

In [7]:
cpp_source = "torch::Tensor rgb_to_grayscale(torch::Tensor image);"

In [ ]:
rgb_to_gs = compile_extension(cuda_source=cuda_source,
                              cpp_source=cpp_source,
                              ext_name='rgb_to_gs',
                              call_funcs=['rgb_to_grayscale'])

In [ ]:
rgb_to_gs.rgb_to_grayscale()

In [ ]:
x = read_image("Grace_Hopper.jpg").permute(1, 2, 0).cuda()
print("mean:", x.float().mean())
print("Input image:", x.shape, x.dtype)

assert x.dtype == torch.uint8

In [ ]:
y = rgb_to_gs.rgb_to_grayscale(x)

print("Output image:", y.shape, y.dtype)
print("mean", y.float().mean())

write_png(y.permute(2, 0, 1).cpu(), "output.png")